In [3]:
!pip install -U pip transformers


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [5]:
checkpoint = 'facebook/nllb-200-distilled-600M'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [6]:
print(f"{len(tokenizer.vocab)}\n")

tokenizer.vocab

256204



{'хим': 111193,
 'льмі': 43428,
 '▁synder': 243420,
 '▁tulet': 179555,
 '▁Nyamenle': 58081,
 'をかけ': 219296,
 '्रांस': 77198,
 '▁dogodilo': 95289,
 'maken': 175079,
 'рӯз': 69671,
 'ledu': 55556,
 '▁kategorija': 199075,
 '▁ಹೌದು': 147998,
 'গৈ': 174026,
 '▁akuk': 122896,
 '▁banyama': 232684,
 'atikweni': 115121,
 'odse': 241680,
 '▁近': 115450,
 'ungannya': 187723,
 'või': 152643,
 'шек': 112469,
 '▁tääu': 174258,
 'ķis': 133681,
 '反対': 51206,
 '▁Че': 43788,
 'ვალის': 166046,
 '▁있겠': 55500,
 '▁transportlīdzek': 106509,
 '▁ክፉ': 150672,
 '▁GI': 156231,
 'ଏସ୍': 242719,
 '▁وچ': 70961,
 'ឡើយ។': 199341,
 '▁[2],': 122497,
 '時間で': 188913,
 'ουργ': 94357,
 '▁gynnig': 220569,
 '▁pasyente': 183453,
 '▁agn': 152843,
 'ство': 3994,
 '▁93;': 43199,
 '▁چینی': 83901,
 '▁कहि': 41305,
 'isiano': 97181,
 '▁adamlar': 36454,
 '▁Ыйсанын': 66123,
 '▁fumaneha': 193841,
 '▁ምክንያት': 66292,
 'лерінің': 50703,
 'zbek': 118981,
 'સી': 27638,
 '▁تاپش': 201854,
 'gida': 170882,
 '▁gyf': 30538,
 'ನ್ನಾಗಿ': 107023,
 'ង្ឃ':

In [7]:
thai_char_min = 0x0E00
thai_char_max = 0x0E7F

thai_tokens = [
    token for token in tokenizer.vocab.keys()
    if any(thai_char_min <= ord(char) <= thai_char_max for char in token)
]

thai_token_count = len(thai_tokens)
sample_size = 20
thai_tokens_sample = thai_tokens[:sample_size]


print(f"{thai_token_count}\n")
for token in thai_tokens_sample:
  print(token)

1712

ไป
ดาว
มาจาก
ชอบ
ดา
้าว
นิ
สร้าง
ลาด
วัน
็ค
แท
ลา
กระ
ปืน
▁ฉันจะ
นอน
น่า
ถ่าย
ตํา


In [8]:
import tensorflow as tf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import math

In [9]:
sentence = 'Work hard, play harder'


In [10]:
cleaned_sentence = sentence.replace(',', '')
cleaned_sentence

'Work hard play harder'

In [11]:
words = cleaned_sentence.split()
words

['Work', 'hard', 'play', 'harder']

In [12]:
sorted_words = sorted(words)
sorted_words

['Work', 'hard', 'harder', 'play']

In [13]:
dc = {word: index for index, word in enumerate(sorted_words)}
dc

{'Work': 0, 'hard': 1, 'harder': 2, 'play': 3}

In [14]:
sentence_int = tf.constant(
    [dc[s] for s in sentence.replace(',', '').split()],
    dtype=tf.int32
)

In [15]:
print(sentence)
print(sentence_int)

Work hard, play harder
tf.Tensor([0 1 3 2], shape=(4,), dtype=int32)


In [16]:
# สร้าง embedding layer
tf.random.set_seed(123)
vocab_size = 50_000
embedding_dim = 2

embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)

In [17]:
embedded_sentence = embed(sentence_int)

In [18]:
embedded_sentence

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.02775565, -0.04259768],
       [ 0.04935188,  0.03270096],
       [ 0.0048217 , -0.04460056],
       [ 0.04521665,  0.0067135 ]], dtype=float32)>

In [19]:
tf.random.set_seed(123)
vocab_size = 50_000
embedding_dim = 2

dummy_input = tf.constant([0, 1, 2], dtype=tf.int32)

# Case 1 Default initializer (RandomUniform(-0.05, 0.05))
embed_default = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
_ = embed_default(dummy_input) # เรียกใช้งาน layer เพื่อสร้าง weights
weights_default = embed_default.get_weights()[0].flatten()
weights_default.shape

(100000,)

In [20]:
# Case 2 GlorotUniform initializer
tf.random.set_seed(123)
embed_glorot = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    embeddings_initializer=tf.keras.initializers.GlorotUniform()
)
_ = embed_glorot(dummy_input) # เรียกใช้งาน layer เพื่อสร้าง weights
weights_glorot = embed_glorot.get_weights()[0].flatten()
weights_glorot.shape

(100000,)

In [21]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Histogram(x=weights_default, nbinsx=50, name="Default Uniform [-0.05, 0.05]", opacity=0.6))
fig.add_trace(go.Histogram(x=weights_glorot, nbinsx=50, name="Glorot Uniform", opacity=0.6))

fig.update_layout(
    title_text='Embedding Layer Initialization Comparison',
    xaxis_title_text='Weight values',
    yaxis_title_text='Frequency',
    barmode='overlay',
    legend_orientation="h",
    legend_yanchor="bottom",
    legend_y=1.02,
    legend_xanchor="right",
    legend_x=1
)

fig.show()

print("Default initializer range ", weights_default.min(), weights_default.max())
print("Glorot initializer range ", weights_glorot.min(), weights_glorot.max())

Default initializer range  -0.049999297 0.04999986
Glorot initializer range  -0.010954167 0.010954126


In [22]:
def glorot_uniform_limits(fan_in, fan_out):
    limit = math.sqrt(6.0 / (fan_in + fan_out))
    a, b = -limit, limit
    return a, b

# ตัวอย่าง Embedding layer (vocab_size=50000, embedding_dim=2)
fan_in = 50000
fan_out = 2

a, b = glorot_uniform_limits(fan_in, fan_out)
print("Glorot Uniform a =", a)
print("Glorot Uniform b =", b)

Glorot Uniform a = -0.010954232067652772
Glorot Uniform b = 0.010954232067652772


In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [24]:
token_embedding_layer = model.model.encoder.embed_tokens
token_embedding_layer.weight.shape

torch.Size([256206, 1024])

In [25]:
long_sentence = "In the vast realm of natural language processing, understanding the nuances of how models handle sequential data is crucial. Positional encoding plays a vital role in providing this essential information to the model, allowing it to differentiate between words at different positions in a sentence, which is fundamental for tasks like translation, summarization, and text generation."

In [26]:
tokens = tokenizer(long_sentence, return_tensors="pt")

print(tokens['input_ids'][0])

tensor([256047,    717,    349,  14430,  12284, 248070,    452,  25307,  65445,
        157278, 248079, 133930,    349,    713,  75831,    452,  11657, 141057,
         47274, 116914, 124785,   6067,    248, 182071, 248075,  12013,  58409,
         12025, 246156,   3054,    705,      9, 104781,  76065,    108, 174693,
          3423, 140515,  18781,    202,    349,  14916, 248079,  82935,     87,
           796,    202,  53054,    502,  25914,  51744,    230,  30158, 199073,
           108,      9, 109267, 248079,   9089,    248,  75529,    351, 226047,
          6399, 200356, 248079,   2493, 109207, 181953, 248079,    540,  35883,
        120531, 248075,      2])


In [27]:
len(tokens['input_ids'][0])

75

In [28]:
token_embedding_layer(tokens['input_ids'][0][0]).shape

torch.Size([1024])

In [29]:
token_embeddings = token_embedding_layer(tokens['input_ids'][0])

print("Token Embedding Matrix shape", token_embeddings.shape)
token_embeddings

Token Embedding Matrix shape torch.Size([75, 1024])


tensor([[-5.0000e+00, -1.2725e+00, -9.3604e-01,  ..., -1.8297e+01,
         -9.1328e+00, -1.0672e+01],
        [ 2.6416e-01,  2.6831e-01,  2.0117e-01,  ...,  3.2715e+00,
         -3.2402e+00,  3.1738e+00],
        [ 4.3579e-01, -2.3352e-01,  2.6825e-02,  ...,  5.4648e+00,
          2.7129e+00,  5.5430e+00],
        ...,
        [ 8.5859e+00, -4.5391e+00, -4.7314e-01,  ..., -7.9529e-02,
          7.4844e+00, -7.5156e+00],
        [-2.4863e+00, -2.7515e-01,  5.6114e-03,  ...,  1.0180e+01,
         -7.2422e+00, -4.8047e+00],
        [-7.8320e-01, -9.0527e-01, -9.4482e-01,  ...,  3.1078e+01,
         -8.1494e-01, -8.7354e-01]], grad_fn=<MulBackward0>)

In [30]:
import plotly.express as px

token_embeddings_np = token_embeddings.detach().numpy()

fig = px.imshow(
    token_embeddings_np,
    color_continuous_scale="RdBu",
    labels=dict(x="Embedding Dimension", y="Token Index", color="Value"),
    title="Token Embedding Heatmap"
)

fig.update_xaxes(side="top")
fig.update_layout(height=500, width=900)
fig.show()

In [31]:
d = embedded_sentence.shape[-1]
d

2

In [32]:
d_q, d_k, d_v = 2, 2, 4

d_q, d_k, d_v

(2, 2, 4)

In [33]:
tf.random.set_seed(123)
W_query = tf.Variable(tf.random.uniform((d, d_q)), trainable=True)
W_key   = tf.Variable(tf.random.uniform((d, d_k)), trainable=True)
W_value = tf.Variable(tf.random.uniform((d, d_v)), trainable=True)

In [34]:
print(W_query.shape, W_key.shape, W_value.shape)


(2, 2) (2, 2) (2, 4)


In [35]:
W_query

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0.12615311, 0.5727513 ],
       [0.2993133 , 0.5461836 ]], dtype=float32)>

In [36]:
W_key


<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[0.88968754, 0.12354946],
       [0.7718717 , 0.6850728 ]], dtype=float32)>

In [37]:
W_value

<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
array([[0.48962688, 0.5857923 , 0.36451697, 0.6550509 ],
       [0.9075084 , 0.37557673, 0.6882372 , 0.25384045]], dtype=float32)>

In [38]:
embedded_sentence

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.02775565, -0.04259768],
       [ 0.04935188,  0.03270096],
       [ 0.0048217 , -0.04460056],
       [ 0.04521665,  0.0067135 ]], dtype=float32)>

In [39]:
queries = tf.matmul(embedded_sentence, W_query)
keys    = tf.matmul(embedded_sentence, W_key)
values  = tf.matmul(embedded_sentence, W_value)

In [40]:
print("Queries shape", queries.shape)
queries

Queries shape (4, 2)


<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.01625151, -0.03916324],
       [ 0.01601373,  0.04612708],
       [-0.01274127, -0.02159846],
       [ 0.00771366,  0.0295647 ]], dtype=float32)>

In [41]:
print("Keys shape", keys.shape)
keys

Keys shape (4, 2)


<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.0575738 , -0.03261171],
       [ 0.0691487 ,  0.02849994],
       [-0.0301361 , -0.02995891],
       [ 0.04541065,  0.01018573]], dtype=float32)>

In [42]:
print("Values shape", values.shape)
values

Values shape (4, 4)


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-0.05224766, -0.03225774, -0.03943471, -0.02899438],
       [ 0.0538404 ,  0.04119167,  0.04049562,  0.04062882],
       [-0.03811455, -0.01392642, -0.02893817, -0.00816297],
       [ 0.02823184,  0.029009  ,  0.02110272,  0.03132336]],
      dtype=float32)>

In [43]:
omega = tf.matmul(queries, keys, transpose_b=True)

print("Omega shape", omega.shape)
print("Omega (Unnormalized attention weights)")
print(omega)

Omega shape (4, 4)
Omega (Unnormalized attention weights)
tf.Tensor(
[[ 0.00221284 -0.00223992  0.00166305 -0.0011369 ]
 [-0.00242625  0.00242195 -0.00186451  0.00119703]
 [ 0.00143793 -0.0014966   0.00103104 -0.00079859]
 [-0.00140826  0.00137598 -0.00111819  0.00065142]], shape=(4, 4), dtype=float32)


In [44]:
d_k = tf.cast(d_k, tf.float32)

scaled_omega = omega / tf.sqrt(d_k)

attention_weights = tf.nn.softmax(scaled_omega, axis=-1)

print("Attention Weights")
print(attention_weights)

Attention Weights
tf.Tensor(
[[0.25036916 0.24958211 0.25027186 0.24977684]
 [0.24960084 0.250458   0.2497     0.25024113]
 [0.25024652 0.2497278  0.25017455 0.2498511 ]
 [0.24977311 0.25026533 0.24982437 0.25013715]], shape=(4, 4), dtype=float32)


In [45]:
row_sums = tf.reduce_sum(attention_weights, axis=-1)

print("Sum of each row in attention_weights")
row_sums

Sum of each row in attention_weights


<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1.        , 0.99999994, 0.99999994, 1.        ], dtype=float32)>

In [46]:
context_vector = tf.matmul(attention_weights, values)

print("Context Vector shape", context_vector.shape)
print(context_vector)

Context Vector shape (4, 4)
tf.Tensor(
[[-0.00213094  0.00596475 -0.0017377   0.00866182]
 [-0.00200874  0.00604704 -0.00164558  0.00873889]
 [-0.00211088  0.00597821 -0.00172258  0.00867441]
 [-0.00203579  0.0060288  -0.00166597  0.0087218 ]], shape=(4, 4), dtype=float32)


In [47]:
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, d_in, d_out_kq, d_out_v):
        super().__init__()
        self.d_out_kq = d_out_kq

        self.W_query = tf.Variable(
            tf.random.uniform((d_in, d_out_kq)), trainable=True
        )
        self.W_key = tf.Variable(
            tf.random.uniform((d_in, d_out_kq)), trainable=True
        )
        self.W_value = tf.Variable(
            tf.random.uniform((d_in, d_out_v)), trainable=True
        )

    def call(self, x):
        keys = tf.matmul(x, self.W_key)      # [T, d_out_kq]
        queries = tf.matmul(x, self.W_query) # [T, d_out_kq]
        values = tf.matmul(x, self.W_value)  # [T, d_out_v]

        # Attention scores: QKᵀ
        attn_scores = tf.matmul(queries, keys, transpose_b=True)  # [T, T]

        # Softmax (scaled by sqrt(d_k))
        attn_weights = tf.nn.softmax(
            attn_scores / tf.math.sqrt(tf.cast(self.d_out_kq, tf.float32)), axis=-1
        )  # [T, T]

        # Weighted sum
        context_vec = tf.matmul(attn_weights, values)  # [T, d_out_v]
        return context_vec

In [48]:
tf.random.set_seed(123)

d_in, d_out_kq, d_out_v = 2, 2, 4

sa = SelfAttention(d_in, d_out_kq, d_out_v)

out = sa(embedded_sentence)

print(out.shape)  # (T, d_out_v)
print(out.numpy())

(4, 4)
[[-0.00213094  0.00596475 -0.0017377   0.00866182]
 [-0.00200874  0.00604704 -0.00164558  0.00873889]
 [-0.00211088  0.00597821 -0.00172258  0.00867441]
 [-0.00203579  0.0060288  -0.00166597  0.0087218 ]]


In [49]:
class MultiHeadAttentionWrapper(tf.keras.layers.Layer):
    def __init__(self, d_in, d_out_kq, d_out_v, num_heads):
        super().__init__()
        self.heads = [
            SelfAttention(d_in, d_out_kq, d_out_v)
            for _ in range(num_heads)
        ]

    def call(self, x):
        # รันทุก head แล้ว concat ตามแกนสุดท้าย
        head_outputs = [head(x) for head in self.heads]   # list of [T, d_out_v]
        return tf.concat(head_outputs, axis=-1)           # [T, num_heads * d_out_v]

In [50]:
tf.random.set_seed(123)

d_in, d_out_kq, d_out_v = 2, 2, 1

sa = SelfAttention(d_in, d_out_kq, d_out_v)

# ถ้า embedded_sentence.shape = [T, d_in] เช่น [6, 3]
out = sa(embedded_sentence)

print(out.shape)   # (T, d_out_v) -> (6, 1)
print(out.numpy())

(4, 1)
[[0.0017261 ]
 [0.00181935]
 [0.00174139]
 [0.0017987 ]]


In [51]:
tf.random.set_seed(123)

mha = MultiHeadAttentionWrapper(
    d_in, d_out_kq, d_out_v, num_heads=3
)

# run MHA
context_vecs = mha(embedded_sentence)   # [T, num_heads * d_out_v]

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tf.Tensor(
[[ 0.0017261  -0.00057679  0.00215093]
 [ 0.00181935 -0.00045881  0.0022944 ]
 [ 0.00174139 -0.00055073  0.00220108]
 [ 0.0017987  -0.00048057  0.00227991]], shape=(4, 3), dtype=float32)
context_vecs.shape: (4, 3)
